In [8]:
%pip install langchain_google_genai load_dotenv crewai crewai_tools langchain_community sentence-transformers langchain-groq langchain_huggingface --quiet

In [11]:
from google.colab import userdata
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
load_dotenv()

from crewai import Agent, Task, Crew
from crewai import LLM
os.environ['GEMINI_API_KEY'] = userdata.get('Gemini_Api')
llm = LLM(model="gemini/gemini-1.5-flash")
# os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API')

TOOL

In [ ]:
from crewai_tools import SerperDevTool
os.environ['SERPER_API_KEY'] = userdata.get('SERPER_API_KEY')
tool = SerperDevTool() ## Uses Live Web Search for searching information

serper_tool= SerperDevTool()
data = serper_tool.run(search_query="Deep learning")
print(data)


AGENT

In [23]:
researcher_paper_scrapper=Agent(
role="Senior Paper Researcher",
goal='Make lists of the top research paper scrapped from web of topic {topic}',

backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowlegde that could change"
    "the world , you have expertise in listing the top research paper of topic {topic},"
    "you are friendly and accurate about you work to list research paper"
),
tools=[tool],
llm=llm,
allow_delegation=True
)


In [24]:
researcher_paper_info=Agent(
role="Senior Paper Researcher information",
goal='Make brief info of the top research paper scrapped from web of topic {topic} by agent researcher_paper_scrapper ',

backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowlegde that could change"
    "the world , you have expertise in making brief summary the top research paper of topic {topic} scrapped by researcher_paper_scrapper,"
    "you are friendly and accurate about you work to make summary research paper"
),
# tools=[tool],
llm=llm,
allow_delegation=True
)

TASK

In [25]:
paper_scraper_task = Task(
    description=("This agent scrapes the research paper enter by user of topic {topic} "
        "selecting the most appropriate answer based on the  real and perfect paper that helps to understand topic more accurately."
        "The agent uses an LLM to "
        "generate the list of that research paper of topic {topic},"
    ),
    expected_output="Give the list of top research paper of topic {topic}",
    agent=researcher_paper_scrapper,
)

In [26]:
paper_info_task = Task(
    description=("This agent makes summary of the research paper enter by user of topic {topic} scrapped by researcher_paper_scrapper agent"
        "selecting the most appropriate answer based on the real and perfect paper that helps to understand topic more accurately."
        "The agent uses an LLM to"
        "generate the summary of that research paper of topic {topic} given by researcher_paper_scrapper agent"
    ),
    expected_output="Give the list of top research paper of topic {topic}",
    agent=researcher_paper_info,
)

In [ ]:
crew = Crew(
  agents=[researcher_paper_scrapper,researcher_paper_info],
  tasks=[paper_scraper_task, paper_info_task],
  verbose=True
)

In [ ]:
inputs = {
    "topic": str(input("Enter Topic name to list research paper \n")),
}
result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)